In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
from keras.preprocessing import image

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip",'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip",'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
train_path='./train'
test_path='./test1'

In [ ]:
filenames=os.listdir('./train')

In [ ]:
categories=[]
for i in filenames:
    category=i.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
    

In [ ]:
train=pd.DataFrame({'file':filenames,'category':categories})

In [ ]:
labels=train['category'].map({1:'dog',0:'cat'})

In [ ]:
k=0

In [ ]:
for i in range(4):
    plt.figure(figsize=(4,4))
    plt.imshow(image.load_img(os.path.join(train_path,filenames[k])))
    plt.title(labels[k])
    k+=1

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,val= train_test_split(train, train_size=0.75)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.5,rotation_range=40,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1/255)

In [ ]:
train['category']=train['category'].map({1:'dog',0:'cat'})

In [ ]:
val['category']=val['category'].map({1:'dog',0:'cat'})

In [ ]:
train.head()

In [ ]:
train_generator=train_datagen.flow_from_dataframe(train, directory=train_path, x_col='file', y_col='category', target_size=(150,150), class_mode='binary', batch_size=32)

In [ ]:
val_generator=test_datagen.flow_from_dataframe(val, directory=train_path,x_col='file', y_col='category', target_size=(150,150), class_mode='binary', batch_size=20)

In [ ]:
from keras.applications import ResNet50, MobileNet

In [ ]:
mnet=MobileNet(input_shape=(150,150,3), include_top=False)

In [ ]:
mnet.trainable=False

In [ ]:
mnet.summary()

In [ ]:
model=tf.keras.models.Sequential([mnet,
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(32, activation='relu'),
                                  tf.keras.layers.Dense(1, activation='sigmoid')
                                 ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [ ]:
train_generator[0][0][0].shape

In [ ]:
history=model.fit(train_generator,epochs=5,validation_data=val_generator,validation_steps=250)

In [ ]:
test_names=os.listdir('./test1')

In [ ]:
test=pd.DataFrame({'file':test_names})

In [ ]:
test_generator=test_datagen.flow_from_dataframe(test, directory=test_path,x_col='file', y_col=None, target_size=(150,150), class_mode=None, batch_size=20, shuffle=False)

In [ ]:
predict=model.predict(test_generator)

In [ ]:
len(predict)

In [ ]:
k=0

In [ ]:
for i,j in zip(test_names[k:k+6],range(6)):
    plt.subplot(2,3,j+1)
    plt.imshow(image.load_img(os.path.join(test_path,i)))
    plt.title(predict[k+j])
k+=4

In [ ]:
pred=[]

In [ ]:
for i in predict:
    if i>0.5:
        pred.append(1)
    else:
        pred.append(0)

In [ ]:
pred=np.asarray(pred)

In [ ]:
pred=pred.reshape(-1,)

In [ ]:
submission=pd.DataFrame()

In [ ]:
ids=[]
for i in test_names:
    ids.append(i.split('.')[0])

In [ ]:
submission['id']=ids

In [ ]:
submission['labels']=pred

In [ ]:
k=0

In [ ]:
for i,j,m in zip(submission['id'][k:k+6],submission['labels'][k:k+6], range(6)):
    plt.subplot(2,3,m+1)
    plt.imshow(image.load_img(os.path.join(test_path,(i+'.jpg'))))
    plt.title(j)
    k+=1

In [ ]:
submission.to_csv('submission.csv', index=False)